In [3]:
import numpy as np
import ray
from ray.util.joblib import register_ray

from joblib import parallel_backend

from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC      

## Data

In [4]:
digits = load_digits()

## Initializations

In [5]:
ray.init(address='auto')

2020-03-31 18:05:32,781	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.


{'node_ip_address': '172.31.25.223',
 'redis_address': '172.31.25.223:6379',
 'object_store_address': '/tmp/ray/session_2020-03-31_16-08-57_185823_3200/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-03-31_16-08-57_185823_3200/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-03-31_16-08-57_185823_3200'}

In [7]:
register_ray()

In [8]:
param_space = {
    'C': np.logspace(-6, 6, 30),
    'gamma': np.logspace(-8, 8, 30),
    'tol': np.logspace(-4, -1, 30),
    'class_weight': [None, 'balanced'],
}

In [9]:
model = SVC(kernel='rbf')
search = RandomizedSearchCV(model, param_space, cv=5, n_iter=300, verbose=1)

In [10]:
with parallel_backend('ray'):
    search.fit(digits.data, digits.target)

2020-03-31 18:05:43,852	WARNING pool.py:340 -- The 'context' argument is not supported using ray. Please refer to the documentation for how to control ray initialization.


Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend RayBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   42.2s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:  5.3min finished
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
2020-03-31 18:11:01,952	WARNING worker.py:1072 -- A worker died or was killed while executing task ffffffffffffffff21937c490700.
2020-03-31 18:11:01,974	WARNING worker.py:1072 -- A worker died or was killed whil